In [1]:
import os
os.chdir('../')
import model.op
from model.op import FleetSizeOptimizer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
kappa = np.array([[0, 41.58, 36.61],[37.31, 0, 33.32], [33.32, 36.61, 0]])/160*100
tau = np.array([[0, 18.71, 15.25], [19.43, 0, 16.65], [16.65, 15.25, 0]])

# kappa = kappa[0:2,0:2]
# tau = tau[0:2,0:2]
op = FleetSizeOptimizer(flight_time=tau, energy_consumption=kappa, schedule='schedule_5min_0612.csv')

In [13]:
op.optimize('HHR_WARNER/charging_and_parking_at_middle_vertiport.txt', charging_station=[False, False, True])

Set parameter MIPGap to value 0.1
Set parameter FeasibilityTol to value 1e-07
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 34968 rows, 551133 columns and 1464372 nonzeros
Model fingerprint: 0xcbb7e34c
Variable types: 0 continuous, 551133 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-07, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 23348 rows and 369579 columns
Presolve time: 0.68s
Presolved: 11620 rows, 181554 columns, 365686 nonzeros
Variable types: 0 continuous, 181554 integer (160 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   43076    1.3410037e+02   0.000000e+00   2.675248e+04      5s
   68810    9.1000

In [14]:
op.parse_result('HHR_WARNER/charging_and_parking_at_middle_vertiport.txt')
op.calculate_aircraft_states()
# op.plot_aircraft_state()

In [15]:
dist = np.array([[0, 30, 15], [30, 0, 15], [15, 15, 0]])
# dist = dist[0:2,0:2]
op.get_summary_statistics(dist)

Fleet size: 91.0
Total number of pads: 129; [14 36 79] 
Total number of flights: 2592.0; demand: 771.0; repositioning: 1821.0
Total energy consumption: 50958.0 kWh
Total aircraft miles: 103260.0 mi
Ratio of revenue aircraft miles to aircraft miles: 0.33599651365485184


# Wind Variation

In [2]:
from tqdm import tqdm

In [3]:
hhr_warner_energy = pd.read_csv('input/route_profile/hhr_warner_energy.csv').to_numpy()[:,1:]
warner_hhr_energy = pd.read_csv('input/route_profile/warner_hhr_energy.csv').to_numpy()[:,1:]
warner_hhr_flight_duration = pd.read_csv('input/route_profile/warner_hhr_flight_duration.csv').to_numpy()[:,1:]
hhr_warner_flight_duration = pd.read_csv('input/route_profile/hhr_warner_flight_duration.csv').to_numpy()[:,1:]

In [25]:
files = os.listdir('output/HHR_WARNER/wind_variation')
files = [file.split('_')[1]+file.split('_')[3] for file in files]

In [26]:
mag = np.arange(0, 60, 10)
angel = np.arange(0, 360+45, 45)
dist = np.array([[0,30],[30,0]])
fleetsize_all = np.zeros((hhr_warner_energy.shape[0], hhr_warner_energy.shape[1]))
num_pads_all = np.zeros((hhr_warner_energy.shape[0], hhr_warner_energy.shape[1]))
vertiport_pads_all = np.zeros((hhr_warner_energy.shape[0], hhr_warner_energy.shape[1], 2))
num_flight_all = np.zeros((hhr_warner_energy.shape[0], hhr_warner_energy.shape[1]))
energy_consumption_all = np.zeros((hhr_warner_energy.shape[0], hhr_warner_energy.shape[1]))
aircraft_miles_all = np.zeros((hhr_warner_energy.shape[0], hhr_warner_energy.shape[1]))
revenue_miles_all = np.zeros((hhr_warner_energy.shape[0], hhr_warner_energy.shape[1]))


for i in tqdm(range(hhr_warner_energy.shape[0])):
    for j in range(hhr_warner_energy.shape[1]):

        if f'mag_{mag[i]}_ang_{angel[j]}.txt' in files:
            continue
        else:
            kappa = np.array([[0, hhr_warner_energy[i,j]], [warner_hhr_energy[i,j], 0]])/160*100
            tau = np.array([[0, hhr_warner_flight_duration[i,j]], [warner_hhr_flight_duration[i,j], 0]])
            op = FleetSizeOptimizer(flight_time=tau, energy_consumption=kappa, schedule='schedule_5min_0612.csv')
            op.optimize(f'HHR_WARNER/wind_variation/mag_{mag[i]}_ang_{angel[j]}.txt', charging_station=[True, True])
            op.parse_result(f'HHR_WARNER/wind_variation/mag_{mag[i]}_ang_{angel[j]}.txt')
            op.calculate_aircraft_states()

            fleetsize, num_pads, vertiport_pads, num_flight, energy_consumption, aircraft_miles, revenue_miles = op.get_summary_statistics(dist, True)

            fleetsize_all[i,j] = fleetsize
            num_pads_all[i,j] = num_pads
            vertiport_pads_all[i,j,:] = vertiport_pads
            num_flight_all[i,j] = num_flight
            energy_consumption_all[i,j] = energy_consumption
            aircraft_miles_all[i,j] = aircraft_miles
            revenue_miles_all[i,j] = revenue_miles




  0%|          | 0/6 [00:00<?, ?it/s]

Set parameter MIPGap to value 0.1
Set parameter FeasibilityTol to value 1e-07
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 21700 rows, 349272 columns and 705872 nonzeros
Model fingerprint: 0x2c5697e1
Variable types: 0 continuous, 349272 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-07, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 1938 rows and 7487 columns
Presolve time: 0.59s
Presolved: 19762 rows, 341785 columns, 692513 nonzeros
Variable types: 0 continuous, 341785 integer (0 binary)
Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.49s

Barrier statistics:
 AA' NZ     : 3.608e+05
 Factor NZ  : 8.409e+06 (roughly 200 MB of memory)
 Factor Ops : 5.

 17%|█▋        | 1/6 [15:34<1:17:54, 934.94s/it]

Fleet size: 33.0
Total number of pads: 41; [14 27] 
Total number of flights: 958.0; demand: 771.0; repositioning: 187.0
Total energy consumption: 11496.0 kWh
Total aircraft miles: 28740.0 mi
Ratio of revenue aircraft miles to aircraft miles: 0.8048016701461378
Set parameter MIPGap to value 0.1
Set parameter FeasibilityTol to value 1e-07
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 21700 rows, 349272 columns and 705872 nonzeros
Model fingerprint: 0x2c5697e1
Variable types: 0 continuous, 349272 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-07, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 1938 rows and 7487 columns
Presolve time: 0.57s
Presolved: 19762 rows, 341785 columns, 692513 nonzeros
Variable types: 0 continuous, 341785 integer (0 binary)
Det

 33%|███▎      | 2/6 [36:07<1:13:59, 1109.90s/it]

...

Fleet size: 31.0
Total number of pads: 44; [15 29] 
Total number of flights: 894.0; demand: 771.0; repositioning: 123.0
Total energy consumption: 10728.0 kWh
Total aircraft miles: 26820.0 mi
Ratio of revenue aircraft miles to aircraft miles: 0.8624161073825504
Set parameter MIPGap to value 0.1
Set parameter FeasibilityTol to value 1e-07
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 21700 rows, 349272 columns and 705872 nonzeros
Model fingerprint: 0x83f191c3
Variable types: 0 continuous, 349272 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-07, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 1938 rows and 7484 columns
Presolve time: 0.56s
Presolved: 19762 rows, 341788 columns, 692516 nonzeros
Variable types: 0 continuous, 341788 integer (0 binary)
Det

 83%|████████▎ | 5/6 [1:47:15<22:06, 1326.84s/it]

Fleet size: 31.0
Total number of pads: 44; [15 29] 
Total number of flights: 894.0; demand: 771.0; repositioning: 123.0
Total energy consumption: 10728.0 kWh
Total aircraft miles: 26820.0 mi
Ratio of revenue aircraft miles to aircraft miles: 0.8624161073825504
Set parameter MIPGap to value 0.1
Set parameter FeasibilityTol to value 1e-07
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 21700 rows, 349272 columns and 705584 nonzeros
Model fingerprint: 0xd4ad1026
Variable types: 0 continuous, 349272 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-07, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 1938 rows and 7485 columns
Presolve time: 0.61s
Presolved: 19762 rows, 341787 columns, 692227 nonzeros
Variable types: 0 continuous, 341787 integer (0 binary)
Det

100%|██████████| 6/6 [2:04:00<00:00, 1240.09s/it]

Fleet size: 31.0
Total number of pads: 43; [14 29] 
Total number of flights: 888.0; demand: 771.0; repositioning: 117.0
Total energy consumption: 11100.0 kWh
Total aircraft miles: 26640.0 mi
Ratio of revenue aircraft miles to aircraft miles: 0.8682432432432432
